In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
from ast import literal_eval

In [3]:
df_mov = pd.read_csv(r"C:\Users\romin\Documents\PROGRAMACION\CIENCIA DE DATOS\HENRY\LABS\PROYECTO INDIVIDUAL\PI 01 MLOps\Dataset\movies_dataset.csv", parse_dates = ['release_date'])

C:\Users\romin\AppData\Local\Temp\ipykernel_18980\2424563618.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_mov = pd.read_csv(r"C:\Users\romin\Documents\PROGRAMACION\CIENCIA DE DATOS\HENRY\LABS\PROYECTO INDIVIDUAL\PI 01 MLOps\Dataset\movies_dataset.csv", parse_dates = ['release_date'])


In [4]:
df_credits = pd.read_csv(r"C:\Users\romin\Documents\PROGRAMACION\CIENCIA DE DATOS\HENRY\LABS\PROYECTO INDIVIDUAL\PI 01 MLOps\Dataset\credits.csv")

In [ ]:
df_credits.shape

In [5]:
duplicadoscred = df_credits[df_credits['id'].duplicated()]
duplicadoscred.shape

(44, 3)

In [6]:
df_credits = df_credits.drop_duplicates(subset = 'id')



join



In [7]:
#convierto columna id al mismo tipo que la del otro df
df_credits['id'] = df_credits['id'].astype(str)

In [8]:
#uso outer para que traiga todo lo de movies, mas lo que tiene en comun con credits
df_movies=df_mov.merge(df_credits, how = 'outer', on = 'id')
df_movies.shape

(45466, 26)



TRANSFORMACIONES



RELLENAR VALORES FALTANTES EN BUDGET Y REVENUE
Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0.

In [9]:
#chequeo si hay valores nulos
print(df_movies['budget'].isnull().sum())
print(df_movies['revenue'].isnull().sum())

0
6


In [10]:
# rellenar con 0 los nulos
df_movies['budget'] = df_movies['budget'].fillna(0)
df_movies['revenue'] = df_movies['revenue'].fillna(0)

In [ ]:
print(type(df_movies['budget'][0]))

In [11]:
#chequeo nulos en Cast y Crew porque pueden interferir al aplanar los diccionarios
print(df_movies['cast'].isnull().sum())
print(df_movies['crew'].isnull().sum())

4
4


In [12]:
# elimino nan de cast y crew
df_movies = df_movies.dropna(subset=['cast'])
df_movies = df_movies.dropna(subset=['crew'])

De haber fechas, deberán tener el formato AAAA-mm-dd, además deberán crear la columna release_year donde extraerán el año de la fecha de estreno.

COLUMNA RELEASE YEAR

In [13]:
# separo la release date en 3, expando las columnas (anio, mes, dia)
df_movies[['release_year','release_month', 'release_day']]=df_movies.release_date.str.split(pat='-',expand=True)

In [ ]:
df_movies.shape

In [14]:
print(type(df_movies['release_year'][0]))

<class 'str'>


In [ ]:
#df_movies.head(2)

In [15]:
#elimino columnas que sobran
del df_movies['release_month']
del df_movies['release_day']

cambio formato de release date

In [16]:
# pasar a formato fecha. los datos que no son strings los convierte en nulos
df_movies['release_date'] = pd.to_datetime(df_movies['release_date'], format="%Y/%m/%d", errors = 'coerce')

In [ ]:
# extraigo solo la fecha, eliminando la hora
#df_movies['release_date'] = df_movies['release_date'].dt.date

elimino nulos de release date

In [17]:
print(df_movies['release_date'].isnull().sum())

87


In [18]:
df_movies = df_movies.dropna(subset=['release_date'])

modifico release year

In [ ]:
# la convierto  al formato de fecha
#df_movies['release_year'] = pd.to_datetime(df_movies['release_year'], format='%Y')
#df_movies['release_year'] = df_movies['release_year'].dt.year

In [ ]:
df_movies.head(2)

Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.

In [19]:
def calcularRetorno(row):
    row['revenue'] = pd.to_numeric(row['revenue'], errors = 'coerce')
    row['budget'] = pd.to_numeric(row['budget'], errors = 'coerce')
    if row['revenue'] == 0 or row['budget'] == 0:
        return 0
    else:
        return round(row['revenue'] / row['budget'],2)

df_movies['return'] = np.where((df_movies['revenue'] == 0) | (df_movies['budget'] == 0), 0, df_movies.apply(calcularRetorno, axis=1))

eliminar columnas que no serán utilizadas

In [20]:
del df_movies['video']
del df_movies['imdb_id']
del df_movies['adult']
del df_movies['original_title']
del df_movies['poster_path']
del df_movies['homepage']

unnesting

In [21]:
def desanidar(obj):
    lista = []
    for i in literal_eval (obj):
        try:
            lista.append(i['name'])
        except (ValueError, TypeError, KeyError):
            pass
    return lista

In [22]:
df_movies['genres'] = df_movies['genres'].apply(desanidar)
df_movies['production_companies'] = df_movies['production_companies'].apply(desanidar)
df_movies['production_countries'] = df_movies['production_countries'].apply(desanidar)
df_movies['spoken_languages'] = df_movies['spoken_languages'].apply(desanidar)
df_movies['cast'] = df_movies['cast'].apply(desanidar)

In [23]:
import ast
def desanidar2(obj):
    if isinstance(obj, str) and '{' in obj:
        dic = ast.literal_eval(obj)
        return dic['name']
    

In [24]:
df_movies['belongs_to_collection'] = df_movies['belongs_to_collection'].apply(desanidar2)

In [25]:
def desanidar3(obj):
    lista = []
    for i in literal_eval (obj):
        if i['job'] == 'Director':
            try:
                lista.append(i['name'])
            except (ValueError, TypeError, KeyError):
                pass
    return lista

In [26]:
df_movies['crew'] = df_movies['crew'].apply(desanidar3)




EXPORTO ARCHIVOS INDIVIDUALES QUE VOY A USAR PARA LA API




In [ ]:
#df fecha estreno
df_estreno = df_movies[['title', 'release_date']]
df_estreno.to_csv(r"C:\Users\romin\Documents\PROGRAMACION\CIENCIA DE DATOS\HENRY\LABS\PROYECTO INDIVIDUAL\PI 01 MLOps\peliculas_por_estreno.csv")


In [ ]:
#df puntuacion
df_puntuacion = df_movies[['title', 'release_year', 'popularity']]
df_puntuacion.to_csv(r"C:\Users\romin\Documents\PROGRAMACION\CIENCIA DE DATOS\HENRY\LABS\PROYECTO INDIVIDUAL\PI 01 MLOps\peliculas_por_popularidad.csv")


In [ ]:
#df votos
df_votos = df_movies[['title', 'release_year', 'vote_count', 'vote_average']]
df_votos.to_csv(r"C:\Users\romin\Documents\PROGRAMACION\CIENCIA DE DATOS\HENRY\LABS\PROYECTO INDIVIDUAL\PI 01 MLOps\peliculas_por_votos.csv")


In [ ]:
#df directores
df_directores = df_movies[['title', 'crew', 'return','release_year','budget', 'revenue']]
df_directores.to_csv(r"C:\Users\romin\Documents\PROGRAMACION\CIENCIA DE DATOS\HENRY\LABS\PROYECTO INDIVIDUAL\PI 01 MLOps\peliculas_por_director.csv")


In [ ]:
#df actores
df_actores = df_movies[['title', 'cast', 'return']]
df_actores.to_csv(r"C:\Users\romin\Documents\PROGRAMACION\CIENCIA DE DATOS\HENRY\LABS\PROYECTO INDIVIDUAL\PI 01 MLOps\peliculas_por_actor.csv")


In [43]:
#df modelo
df_movies = pd.read_csv(r"C:\Users\romin\Documents\PROGRAMACION\CIENCIA DE DATOS\HENRY\LABS\PROYECTO INDIVIDUAL\PI 01 MLOps\df_movies_final.csv")
campos_mod = ['title', 'genres', 'id', 'release_year', 'runtime', 'popularity', 'cast', 'crew']
df_modelo = df_movies[campos_mod]
columnas = ['popularity', 'release_year']  # Nombres de las columnas por las cuales deseas ordenar
df_modelo = df_modelo.sort_values(by=columnas, ascending=False)
df_modelo = df_modelo[(df_modelo['runtime'] == 0.0) | (df_modelo['runtime']>40)]
df_modelo['genres'] = df_modelo['genres'].str.replace("[\[\]',]", "", regex=True)
df_modelo['cast'] = df_modelo['cast'].str.replace("[\[\]',]", "", regex=True)
df_modelo['crew'] = df_modelo['crew'].str.replace("[\[\]',]", "", regex=True)
df_modelo = df_modelo.head(6000)
df_modelo.reset_index(drop=True, inplace=True)
df_modelo.reset_index(inplace=True)
df_modelo
#df_modelo.to_csv(r"C:\Users\romin\Documents\PROGRAMACION\CIENCIA DE DATOS\HENRY\LABS\PROYECTO INDIVIDUAL\PI 01 MLOps\modelo.csv")


,index,title,genres,id,release_year,runtime,popularity,cast,crew
0,0,Minions,Family Animation Adventure Comedy,211672,2015,91.0,547.488298,Sandra Bullock Jon Hamm Michael Keaton Allison...,Kyle Balda Pierre Coffin
1,1,Wonder Woman,Action Adventure Fantasy,297762,2017,141.0,294.337037,Gal Gadot Chris Pine Robin Wright Danny Huston...,Patty Jenkins
2,2,Beauty and the Beast,Family Fantasy Romance,321612,2017,129.0,287.253654,Emma Watson Dan Stevens Luke Evans Kevin Kline...,Bill Condon
3,3,Baby Driver,Action Crime,339403,2017,113.0,228.032744,Ansel Elgort Lily James Kevin Spacey Jamie Fox...,Edgar Wright
4,4,Big Hero 6,Adventure Family Animation Action Comedy,177572,2014,102.0,213.849907,Scott Adsit Ryan Potter Daniel Henney T.J. Mil...,Chris Williams Don Hall
...,...,...,...,...,...,...,...,...,...
5995,5995,Angels Sing,TV Movie Family Drama,57210,2013,87.0,7.073059,Chandler Canterbury Willie Nelson Harry Connic...,Tim McCanlies
5996,5996,Pokémon 4Ever: Celebi - Voice of the Forest,Adventure Fantasy Animation Science Fiction Fa...,12600,2001,75.0,7.072301,Veronica Taylor Rachael Lillis Maddie Blaustei...,Kunihiko Yuyama Jim Malone
5997,5997,Sleep Tight,Thriller,77495,2011,107.0,7.071300,Luis Tosar Marta Etura Alberto San Juan Petra...,Jaume Balagueró
5998,5998,El Dorado,Western,6644,1966,120.0,7.070669,John Wayne Robert Mitchum James Caan Charlene ...,Howard Hawks


In [45]:
df_modelo.to_csv(r"C:\Users\romin\Documents\PROGRAMACION\CIENCIA DE DATOS\HENRY\LABS\PROYECTO INDIVIDUAL\PI 01 MLOps\modelo.csv")


In [29]:
df_modelo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45375 entries, 0 to 45465
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         45375 non-null  object 
 1   genres        45375 non-null  object 
 2   id            45375 non-null  object 
 3   release_year  45375 non-null  object 
 4   runtime       45129 non-null  float64
 5   popularity    45375 non-null  object 
 6   cast          45375 non-null  object 
 7   crew          45375 non-null  object 
dtypes: float64(1), object(7)
memory usage: 3.1+ MB


In [ ]:
# creo y exporto crew
crew = df_movies[['crew', 'id']]
crew.to_csv(r'C:\Users\romin\Documents\PROGRAMACION\CIENCIA DE DATOS\HENRY\LABS\PROYECTO INDIVIDUAL\PI 01 MLOps\crew_final.csv')


In [ ]:
#exporto csv completo limpio
df_movies.to_csv(r'C:\Users\romin\Documents\PROGRAMACION\CIENCIA DE DATOS\HENRY\LABS\PROYECTO INDIVIDUAL\PI 01 MLOps\df_movies_final.csv')
